Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


## Utility functions

In [4]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.05))

def create_biases(shape):
    return tf.Variable(tf.zeros(shape = shape))

In [5]:
def fully_connected(input_layer, n_outputs, use_relu = False):
    n_inputs = input_layer.get_shape()[1]
    
    weights = create_weights(shape = (n_inputs, n_outputs))
    biases = create_biases(shape = (n_outputs))

    layer = tf.matmul(input_layer, weights)
    layer = tf.nn.bias_add(layer, biases)
    
    if use_relu:
        layer = tf.nn.relu(layer)
        
    return layer

In [6]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [7]:
class Network(object):
    def __init__(self, session, input_size, output_size):
        # TF session
        self.session = session
        
        # Define placeholders
        self.input_ph = tf.placeholder(tf.float32, shape=(None, input_size))
        self.output_ph = tf.placeholder(tf.float32, shape=(None, output_size))
        self.dropout_ph = tf.placeholder(tf.float32)
        self.dropout_keep_p = 1.0
        
        # Define must-haves
        self.optimizer = None
        self.loss = None
        self.prediction = None
                
    def train(self, X_train, y_train, X_val, y_val):
        # Define training parameters
        batch_size = 128
        n_epochs = 3000
        
        # Initialize variables
        self.session.run(tf.global_variables_initializer())
        
        
        for epoch in range(n_epochs):
            # Shuffle data
            shuffle(X_train, y_train)
            
            # Create mini-batch
            offset = (epoch * batch_size) % (X_train.shape[0] - batch_size)
            x_batch = X_train[offset:(offset + batch_size)]
            y_batch = y_train[offset:(offset + batch_size)]
    
            # Run optimizer
            feed_dict = {self.input_ph : x_batch, self.output_ph : y_batch, self.dropout_ph: self.dropout_keep_p}
            _, l, predictions = self.session.run([self.optimizer, self.loss, self.prediction],
                                                 feed_dict=feed_dict)

            if (epoch % 500 == 0):
              print("Minibatch loss at epoch %d: %f" % (epoch, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, y_batch))

              validation_predictions = self.session.run(self.prediction,
                                                        feed_dict={self.input_ph: X_val, self.output_ph: y_val,
                                                                   self.dropout_ph: 1.0})
              print("Validation accuracy: %.1f%%" % accuracy(validation_predictions, y_val))
    
    def evaluate(self, X_test, y_test):
        test_predictions = self.session.run(self.prediction,
                                            feed_dict={self.input_ph: X_test, self.output_ph: y_test,
                                                       self.dropout_ph: 1.0})
        print("Test accuracy: %.1f%%" % accuracy(test_predictions, y_test))

In [8]:
class Perceptron(Network):
    def __init__(self, session, input_size, output_size, l2 = 0.0):
        super(Perceptron, self).__init__(session, input_size, output_size)
        
        # Construct network
        w = create_weights(shape = (input_size, output_size))
        b = create_biases(shape = (output_size))
        
        logits = tf.matmul(self.input_ph, w)
        logits = tf.nn.bias_add(logits, b)
        
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, self.output_ph))

        # Add regularization
        self.loss = self.loss + l2 * tf.nn.l2_loss(w)
        
        # Define outputs
        self.optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(self.loss)
        self.prediction = tf.nn.softmax(logits)

In [9]:
class OneHiddenLayerNetwork(Network):
    def __init__(self, session, input_size, output_size, l2 = 0.0, dropout_keep_p = 1.0):
        super(OneHiddenLayerNetwork, self).__init__(session, input_size, output_size)
        
        # Define dropout placeholder
        self.dropout_keep_p = dropout_keep_p
        
        # Construct network
        # First layer
        size1 = 1024
        w1 = create_weights(shape = (input_size, size1))
        b1 = create_biases(shape = (size1))
        
        logits = tf.matmul(self.input_ph, w1)
        logits = tf.nn.bias_add(logits, b1)
        
        logits = tf.nn.relu(logits)
        
        logits = tf.nn.dropout(logits, self.dropout_ph)
        
        # Second layer
        size2 = output_size
        
        w2 = create_weights(shape = (size1, size2))
        b2 = create_biases(shape = (size2))
        
        logits = tf.matmul(logits, w2)
        logits = tf.nn.bias_add(logits, b2)
        
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, self.output_ph))

        # Add regularization
        self.loss = self.loss + l2 * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
        
        # Define outputs
        self.optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(self.loss)
        self.prediction = tf.nn.softmax(logits)

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
with tf.Session() as session:
    logistic_network = Perceptron(session,
                                  input_size = train_dataset.shape[1],
                                  output_size = train_labels.shape[1],
                                  l2 = 0.01)
    logistic_network.train(train_dataset, train_labels, valid_dataset, valid_labels)
    logistic_network.evaluate(test_dataset, test_labels)

Minibatch loss at epoch 0: 2.527616
Minibatch accuracy: 8.6%
Validation accuracy: 42.5%
Minibatch loss at epoch 500: 0.662686
Minibatch accuracy: 83.6%
Validation accuracy: 81.0%
Minibatch loss at epoch 1000: 0.883597
Minibatch accuracy: 77.3%
Validation accuracy: 81.4%
Minibatch loss at epoch 1500: 0.621865
Minibatch accuracy: 85.2%
Validation accuracy: 80.5%
Minibatch loss at epoch 2000: 0.725370
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
Minibatch loss at epoch 2500: 0.824558
Minibatch accuracy: 82.0%
Validation accuracy: 81.1%
Test accuracy: 88.5%


In [11]:
with tf.Session() as session:
    neural_network = OneHiddenLayerNetwork(session,
                                          input_size = train_dataset.shape[1],
                                          output_size = train_labels.shape[1],
                                          l2 = 0.01)
    neural_network.train(train_dataset, train_labels, valid_dataset, valid_labels)
    neural_network.evaluate(test_dataset, test_labels)

Minibatch loss at epoch 0: 10.343893
Minibatch accuracy: 12.5%
Validation accuracy: 36.6%
Minibatch loss at epoch 500: 0.744336
Minibatch accuracy: 82.8%
Validation accuracy: 83.3%
Minibatch loss at epoch 1000: 0.817789
Minibatch accuracy: 80.5%
Validation accuracy: 83.2%
Minibatch loss at epoch 1500: 0.640281
Minibatch accuracy: 85.9%
Validation accuracy: 83.2%
Minibatch loss at epoch 2000: 0.720172
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at epoch 2500: 0.756158
Minibatch accuracy: 87.5%
Validation accuracy: 83.2%
Test accuracy: 90.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [12]:
with tf.Session() as session:
    n_samples = 512
    logistic_network = Perceptron(session,
                                  input_size = train_dataset.shape[1],
                                  output_size = train_labels.shape[1],
                                  l2 = 0.0)
    logistic_network.train(train_dataset[0:n_samples], train_labels[0:n_samples],
                           valid_dataset, valid_labels)
    logistic_network.evaluate(test_dataset, test_labels)

Minibatch loss at epoch 0: 2.467968
Minibatch accuracy: 7.8%
Validation accuracy: 45.4%
Minibatch loss at epoch 500: 0.022024
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at epoch 1000: 0.009938
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at epoch 1500: 0.005737
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at epoch 2000: 0.005356
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at epoch 2500: 0.003934
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Test accuracy: 82.8%


In [ ]:
with tf.Session() as session:
    n_samples = 512
    logistic_network = Perceptron(session,
                                  input_size = train_dataset.shape[1],
                                  output_size = train_labels.shape[1],
                                  l2 = 0.05)
    logistic_network.train(train_dataset[0:n_samples], train_labels[0:n_samples],
                           valid_dataset, valid_labels)
    logistic_network.evaluate(test_dataset, test_labels)

Minibatch loss at epoch 0: 2.730868
Minibatch accuracy: 10.2%
Validation accuracy: 41.4%
Minibatch loss at epoch 500: 0.716466
Minibatch accuracy: 93.8%
Validation accuracy: 76.2%
Minibatch loss at epoch 1000: 0.721337
Minibatch accuracy: 93.0%
Validation accuracy: 76.9%
Minibatch loss at epoch 1500: 0.666911
Minibatch accuracy: 97.7%
Validation accuracy: 73.9%
Minibatch loss at epoch 2000: 0.715113
Minibatch accuracy: 93.8%
Validation accuracy: 76.4%
Minibatch loss at epoch 2500: 0.721643
Minibatch accuracy: 93.0%
Validation accuracy: 77.0%
Test accuracy: 83.7%


We observe that with l2 = 0.05 we obtain a slightly better accuracy in test set! From 82.7% to 83.7%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
with tf.Session() as session:
    n_samples = 512

    neural_network = OneHiddenLayerNetwork(session,
                                          input_size = train_dataset.shape[1],
                                          output_size = train_labels.shape[1],
                                          dropout_keep_p = 1.0)
    neural_network.train(train_dataset[0:n_samples], train_labels[0:n_samples],
                         valid_dataset, valid_labels)
    neural_network.evaluate(test_dataset, test_labels)

Minibatch loss at epoch 0: 2.584520
Minibatch accuracy: 6.2%
Validation accuracy: 37.0%
Minibatch loss at epoch 500: 0.002095
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at epoch 1000: 0.001065
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at epoch 1500: 0.000541
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at epoch 2000: 0.000425
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at epoch 2500: 0.000374
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%


In [ ]:
with tf.Session() as session:
    n_samples = 512

    neural_network = OneHiddenLayerNetwork(session,
                                          input_size = train_dataset.shape[1],
                                          output_size = train_labels.shape[1],
                                          dropout_keep_p = 0.5)
    neural_network.train(train_dataset[0:n_samples], train_labels[0:n_samples],
                         valid_dataset, valid_labels)
    neural_network.evaluate(test_dataset, test_labels)

We observe that with dropout of 0.5 we increase the test accuracy from 85.8% to 86.2%, slightly improving the overfitting.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
